In [1]:
import pandas as pd
import numpy as np
import re

# Para visualizar el progreso:
import tqdm

In [2]:
df_notino_0_25 = pd.read_csv("datos_perfumes_notino_0_25.csv")
df_notino_25_50 = pd.read_csv("datos_perfumes_notino_25_50.csv")
df_notino_50_75 = pd.read_csv("datos_perfumes_notino_50_75.csv")
df_notino_75_100 = pd.read_csv("datos_perfumes_notino_75_100.csv")
df_notino_100_125 = pd.read_csv("datos_perfumes_notino_100_125.csv")
df_notino_125_175 = pd.read_csv("datos_perfumes_notino_125_175.csv")
df_notino_175_200 = pd.read_csv("datos_perfumes_notino_175_200.csv")
df_notino_200_225 = pd.read_csv("datos_perfumes_notino_200_225.csv")
df_notino_225_226 = pd.read_csv("datos_perfumes_notino_225_226.csv")
df_notino_226_248 = pd.read_csv("datos_perfumes_notino_226_248.csv")

In [3]:
df_notino = pd.concat([df_notino_0_25, df_notino_25_50, df_notino_50_75, df_notino_75_100, df_notino_100_125, df_notino_125_175, df_notino_175_200, df_notino_226_248], axis = 0)

In [4]:
df_notino_clean = df_notino.copy()

### Elimino la primera y última columna:

In [5]:
def convertir_a_minusculas(valor):
    if isinstance(valor, str):
        return valor.lower()
    return valor

df_notino_clean = df_notino_clean.map(convertir_a_minusculas)

### Añado la columna tienda:

In [6]:
df_notino_clean.insert(0, 'tienda', "notino")

### Paso las columnas a minusculas:

In [7]:
df_notino_clean = df_notino_clean.map(lambda x: x.lower() if isinstance(x, str) else x)

C:\Users\regue\AppData\Local\Temp\ipykernel_55136\1907073959.py:1: FutureWarning: DataFrame.applymap has been deprecated. Use DataFrame.map instead.
  df_notino_clean = df_notino_clean.applymap(lambda x: x.lower() if isinstance(x, str) else x)


### Limpio categoria

In [8]:
df_notino_clean["categoria"].unique()

array(['perfumes de hombre', 'perfumes de mujer'], dtype=object)

In [9]:
df_notino_clean['categoria'] = df_notino_clean['categoria'].str.replace('perfumes de hombre', 'hombre')
df_notino_clean['categoria'] = df_notino_clean['categoria'].str.replace('perfumes de mujer', 'mujer')

In [10]:
df_notino_clean.head(3)

,tienda,categoria,marca,nombre,tipo,precio_sin_desc,descuento,precio_con_desc,valoracion,num_valoraciones
0,notino,hombre,hugo boss,boss the scent absolute,eau de parfum para hombre,"134,00€",-15 %,"45,40",5,3
1,notino,hombre,versace,eros,eau de toilette para hombre,es el precio con desc,-15 %,"57,50",sin valoracion,sin valoraciones
2,notino,hombre,versace,pour homme,eau de toilette para hombre,es el precio con desc,-15 %,"47,00",sin valoracion,sin valoraciones


### Limpiamos la columna "tipo" agrupando:

In [11]:
def mapear_tipos_y_limpiar(df):

    mapping = {"para hombre"             : "",
    "para mujer"                         : "",
    "ml"                                 : "",
    "0"                                  : "",
    "1"                                  : "",
    "2"                                  : "",
    "3"                                  : "",
    "4"                                  : "",
    "5"                                  : "",
    "6"                                  : "",
    "7"                                  : "",
    "8"                                  : "",
    "9"                                  : "",
    "roll-on"                            : "",
    "con vaporizador para hombre"        : "",
    "recargable"                         : "",
    "recambio"                           : "",
    "(intense)"                          : "",
    "unisex"                             : "",
    "new design para hombre"             : "",
    "(edición limitada)"                 : "",
    "(roll-on)"                          : "",
    "perfumado"                          : "",
    "y+"                                 : "",
    "sin pulverizador"                   : "",
    "extracto de perfume"                : "parfum",
    "en caja"                            : "",
    "new design"                         : "",
    "edición limitada"                   : "",
    "perfume"                            : "parfum",
    "con efecto energizante"             : "",
    "con vaporizador"                    : "",
    "recarga con pulverizador  x"        : "",
    "(x  + x recarga)  x"                : "",
    "agua de colonia"                    : "eau de cologne"
    }
    for name, replacement in mapping.items():
        df['tipo'] = df['tipo'].str.replace(name, replacement, regex=False)

    df['tipo'] = df['tipo'].str.strip()
    
    return df

In [ ]:
df_notino_clean = mapear_tipos_y_limpiar(df_notino_clean)

In [12]:
df_notino_clean["tipo"].value_counts()

tipo
eau de parfum                         4760
eau de toilette                        311
parfum                                 128
eau de cologne                          47
spray corporal                          42
aceite                                  30
lote de regalo                           8
set                                      6
bruma perfumada corporal y capilar       5
sin categoria                            4
desodorante y spray corporal             3
eau fraiche                              2
spray corporal estimulante               2
spray corporal  con purpurina            2
aceite   (roll on)                       2
rocío corporal                           1
spray corporal refrescante               1
aceite  sin alcohol                      1
                                         1
bruma para cabello y cuerpo              1
Name: count, dtype: int64

### Limpio la columna valoración y num_valoraciones

In [13]:
columns = ['valoracion', 'num_valoraciones']

for col in columns:
    df_notino_clean[col] = df_notino_clean[col].replace(['sin valoracion', 'sin valoraciones'], np.nan).astype(float)

In [14]:
df_notino_clean.info()

<class 'pandas.core.frame.DataFrame'>
Index: 5357 entries, 0 to 551
Data columns (total 10 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   tienda            5357 non-null   object 
 1   categoria         5357 non-null   object 
 2   marca             5357 non-null   object 
 3   nombre            5357 non-null   object 
 4   tipo              5357 non-null   object 
 5   precio_sin_desc   5357 non-null   object 
 6   descuento         5357 non-null   object 
 7   precio_con_desc   5357 non-null   object 
 8   valoracion        3798 non-null   float64
 9   num_valoraciones  3798 non-null   float64
dtypes: float64(2), object(8)
memory usage: 460.4+ KB


In [15]:
df_notino_clean

,tienda,categoria,marca,nombre,tipo,precio_sin_desc,descuento,precio_con_desc,valoracion,num_valoraciones
0,notino,hombre,hugo boss,boss the scent absolute,eau de parfum,"134,00€",-15 %,"45,40",5.0,3.0
1,notino,hombre,versace,eros,eau de toilette,es el precio con desc,-15 %,"57,50",NaN,NaN
2,notino,hombre,versace,pour homme,eau de toilette,es el precio con desc,-15 %,"47,00",NaN,NaN
3,notino,hombre,sin marca,sin nombre,sin categoria,es el precio con desc,sin descuento,sin precio,NaN,NaN
4,notino,hombre,narciso rodriguez,for him bleu noir,eau de parfum,"110,00€",-15 %,"57,50",5.0,1.0
...,...,...,...,...,...,...,...,...,...,...
547,notino,mujer,beyoncé,heat rush,eau de toilette,es el precio con desc,sin descuento,sin precio,NaN,NaN
548,notino,mujer,katy perry,katy perry's mad potion,eau de parfum,es el precio con desc,sin descuento,sin precio,NaN,NaN
549,notino,mujer,biotherm,l’eau,eau de toilette,es el precio con desc,sin descuento,sin precio,NaN,NaN
550,notino,mujer,eisenberg,rouge et noir,eau de parfum,es el precio con desc,sin descuento,sin precio,NaN,NaN


### Limpio las columnas precio_sin_desc, precio_con_desc y  descuento:

Al scrapear esta página no se pudo obtener el precio sin descuento en todos los productos debido al diseño de la página. Solo en los que tenían precio con descuento y precio sin descuento se podían sacar bien los datos. En los productos que no tenían descuento el precio sin descuento era el precio_con_desc, por lo que este hay que pasarlo a la columna de precio_sin_desc

- Precios:

In [16]:
 def clean_prices(df, columns):
    for col in columns:
        
        df[col] = df[col].str.replace('[^0-9,]', '', regex=True) 
        df[col] = df[col].str.replace(',', '.', regex=True)
        df[col] = df[col].replace('', '0', regex=True)
        df[col] = df[col].astype(float)  

columns = ['precio_sin_desc', 'precio_con_desc', "descuento"]

clean_prices(df_notino_clean, columns)

In [17]:
df_notino_clean.head(3)

,tienda,categoria,marca,nombre,tipo,precio_sin_desc,descuento,precio_con_desc,valoracion,num_valoraciones
0,notino,hombre,hugo boss,boss the scent absolute,eau de parfum,134.0,15.0,45.4,5.0,3.0
1,notino,hombre,versace,eros,eau de toilette,0.0,15.0,57.5,NaN,NaN
2,notino,hombre,versace,pour homme,eau de toilette,0.0,15.0,47.0,NaN,NaN


In [18]:
# Función para actualizar Con Descuento
def actualizar_valor(row):
    if row['precio_sin_desc'] == 0.0:
        return row['precio_con_desc']
    else:
        return row['precio_sin_desc']

In [19]:
df_notino_clean['precio_sin_desc'] = df_notino_clean.apply(actualizar_valor, axis=1)

- Descuento:

In [20]:
df_notino_clean['descuento'] = (((df_notino_clean['precio_sin_desc'] - df_notino_clean['precio_con_desc']) / df_notino_clean['precio_sin_desc']) * 100).round(2)

In [21]:
df_notino_clean.head(3)

,tienda,categoria,marca,nombre,tipo,precio_sin_desc,descuento,precio_con_desc,valoracion,num_valoraciones
0,notino,hombre,hugo boss,boss the scent absolute,eau de parfum,134.0,66.12,45.4,5.0,3.0
1,notino,hombre,versace,eros,eau de toilette,57.5,0.00,57.5,NaN,NaN
2,notino,hombre,versace,pour homme,eau de toilette,47.0,0.00,47.0,NaN,NaN


### Elimino datos sin valores

In [22]:
df_notino_clean.isna().sum()

tienda                 0
categoria              0
marca                  0
nombre                 0
tipo                   0
precio_sin_desc        0
descuento            292
precio_con_desc        0
valoracion          1559
num_valoraciones    1559
dtype: int64

In [23]:
df_notino_clean[df_notino_clean["descuento"].isna()].head(3)

,tienda,categoria,marca,nombre,tipo,precio_sin_desc,descuento,precio_con_desc,valoracion,num_valoraciones
3,notino,hombre,sin marca,sin nombre,sin categoria,0.0,NaN,0.0,NaN,NaN
29,notino,hombre,sin marca,sin nombre,sin categoria,0.0,NaN,0.0,NaN,NaN
192,notino,hombre,al haramain,amber oud tobacco edition,eau de parfum,0.0,NaN,0.0,NaN,NaN


In [24]:
df_notino_clean = df_notino_clean.dropna(subset= "descuento")

In [25]:
df_notino_clean[df_notino_clean["marca"]=="sin marca"]

,tienda,categoria,marca,nombre,tipo,precio_sin_desc,descuento,precio_con_desc,valoracion,num_valoraciones


### Elimino duplicados:

In [26]:
df_notino_clean.duplicated().value_counts()

True     4180
False     885
Name: count, dtype: int64

In [27]:
df_notino_clean = df_notino_clean.drop_duplicates()

### Filtro por los tipos sobre los que vamos a trabajar:

In [202]:
lista_tipos = ['eau de parfum', 'eau de toilette', 'parfum', 'eau de cologne']
df_notino_clean = df_notino_clean[df_notino_clean["tipo"].isin(lista_tipos)].reset_index(drop=True)

In [204]:
df_notino_clean.to_csv("datos_perfumes_notino_filtrado.csv", index= False, sep= ",")